In [ ]:
# Import LabelEncoder
from sklearn.preprocessing import LabelEncoder

# Fill missing values with 0
df.LotFrontage = df.LotFrontage.fillna(0)

# Create a boolean mask for categorical columns
categorical_mask = (df.dtypes == object)

# Get list of categorical column names
categorical_columns = df.columns[categorical_mask].tolist()
categorical_columns=['MSZoning', 'PavedDrive', 'Neighborhood', 'BldgType', 'HouseStyle']

# Print the head of the categorical columns
print(df[categorical_columns].head())

# Create LabelEncoder object: le
le = LabelEncoder()

# Apply LabelEncoder to categorical columns
df[categorical_columns] = df[categorical_columns].apply(lambda x: le.fit_transform(x))

# Print the head of the LabelEncoded categorical columns
print(df[categorical_columns].head())

# Import necessary modules
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline

# Fill LotFrontage missing values with 0
X.LotFrontage = X.LotFrontage.fillna(0)

# Setup the pipeline steps: steps
steps = [("ohe_onestep", DictVectorizer(sparse=False)),
         ("xgb_model", xgb.XGBRegressor())]

# Create the pipeline: xgb_pipeline
xgb_pipeline = Pipeline(steps)

# Fit the pipeline
xgb_pipeline.fit(X.to_dict("records"),y)

In [ ]:
expected_profit
<dbl>
expected_ROME
<dbl>
auc
<dbl>
model
<chr>
38172.12	2.04	0.7660188	professor_model_profit

In [ ]:
# Create arrays for the features and the target: X, y
X, y = churn_data.iloc[:,:-1], churn_data.iloc[:,-1]

# Create the DMatrix from X and y: churn_dmatrix
churn_dmatrix = xgb.DMatrix(data=X, label=y)

# Create the parameter dictionary: params
params = {"objective":"reg:logistic", "max_depth":3}

# Perform cross_validation: cv_results
cv_results = xgb.cv(dtrain=churn_dmatrix, params=params, 
                  nfold=3, num_boost_round=5, 
                  metrics="auc", as_pandas=True, seed=123)

# Print cv_results
print(cv_results)

# Print the AUC
print((cv_results["test-auc-mean"]).iloc[-1])

In [ ]:
# Create the parameter grid: gbm_param_grid
gbm_param_grid = {
    'colsample_bytree': [0.3, 0.7],
    'n_estimators': [50],
    'max_depth': [2, 5]
}

# Instantiate the regressor: gbm
gbm = xgb.XGBRegressor()

# Perform grid search: grid_mse
grid_mse = GridSearchCV(param_grid=gbm_param_grid,estimator=gbm,scoring="neg_mean_squared_error",cv=4,verbose=1)


# Fit grid_mse to the data
grid_mse.fit(X,y)

# Print the best parameters and lowest RMSE
print("Best parameters found: ", grid_mse.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(grid_mse.best_score_)))

In [ ]:
data = pd.read_csv('../intuit75k_withstate.csv')
data['label']=1
data.loc[data.res1 == 'No','label'] = 0

categorical_columns=['zip_state','sex']
othercol=['numords','last','dollars','sincepurch','bizflag','owntaxprod','version1','upgraded','training']
keep=categorical_columns+othercol+['label']
combind_data=data.loc[:,keep]
combind_data[categorical_columns] = combind_data[categorical_columns].apply(lambda x: LabelEncoder().fit_transform(x))
X_train=combind_data.loc[combind_data.training==1].drop(columns='label').drop(columns='training')
y_train=combind_data.loc[combind_data.training==1].label
X_test=combind_data.loc[combind_data.training==0].drop(columns='label').drop(columns='training')
y_test=combind_data.loc[combind_data.training==0].label
Xs = np.concatenate((X_train, X_test), axis=0)
# Create the DMatrix from X and y: churn_dmatrix
train_dmatrix = xgb.DMatrix(data=X_train, label=y_train)

# Create the parameter dictionary: params
params = {"objective":"reg:logistic", "max_depth":6}

# Perform cross_validation: cv_results
cv_results = xgb.cv(dtrain=train_dmatrix, params=params, 
                  nfold=5, num_boost_round=20, 
                  metrics="auc", as_pandas=True, seed=123,
                   early_stopping_rounds=10)

# Print cv_results
print(cv_results)

# Print the AUC
print((cv_results["test-auc-mean"]).iloc[-1])

test_dmatrix=xgb.DMatrix(data=X_test)

#cv_res.shape[0]为最佳迭代次数
bst = xgb.train(dtrain=train_dmatrix, params=params,num_boost_round=cv_results.shape[0])
y_pre = bst.predict(test_dmatrix)

y_pre
fpr, tpr, thresholds = metrics.roc_curve(y_test.values, y_pre)
auc_rf = metrics.auc(fpr, tpr)
auc_rf

#tuning eta
eta_vals = [0.001, 0.01, 0.1]
best_auc = []

for curr_val in eta_vals:

    params["eta"] = curr_val
    
    # Perform cross-validation: cv_results
    cv_results = xgb.cv(dtrain=train_dmatrix, params=params, 
                  nfold=5, num_boost_round=20, 
                  metrics="auc", as_pandas=True, seed=123,
                   early_stopping_rounds=10)
    best_auc.append(cv_results["test-auc-mean"].tail().values[-1])
    
print(pd.DataFrame(list(zip(eta_vals, best_auc)), columns=["eta","best_auc"]))

# Create the parameter grid: gbm_param_grid
gbm_param_grid = {
    'n_estimators': [10,15,20,25], ### how to set n_estimators?
    'max_depth': range(2,10)
}

# Instantiate the regressor: gbm
gbm = xgb.XGBClassifier()

# Perform grid search: grid_auc
randomized_auc = RandomizedSearchCV(
    param_distributions=gbm_param_grid,estimator=gbm,scoring="roc_auc",n_iter=50,cv=5,verbose=1
)

# Fit grid_mse to the data
randomized_auc.fit(X_train,y_train)

print("Best parameters found: ", randomized_auc.best_params_)
print("higest auc found: ", np.abs(randomized_auc.best_score_))

preds =randomized_auc.predict_proba(X_test)
fpr, tpr, thresholds = metrics.roc_curve(y_test.values, preds[:, 1])
auc_rf = metrics.auc(fpr, tpr)
auc_rf
xg_cl = xgb.XGBClassifier(objective='binary:logistic', n_estimators=300, max_depth=4)
xg_cl.fit(X_train,y_train)
preds = xg_cl.predict_proba(X_test)
fpr, tpr, thresholds = metrics.roc_curve(y_test.values, preds[:, 1])
auc_rf = metrics.auc(fpr, tpr)
auc_rf
margin=60
cost=1.41
breakeven_rate=cost/margin

testdata=combind_data.loc[combind_data.training==0]
testdata['xgboost']=preds[:, 1]
testdata['pred_click']=1
testdata.loc[testdata['xgboost']<breakeven_rate,'pred_click']=0
tp=testdata.loc[testdata.pred_click==1].loc[testdata.label==1]
revenue=len(tp)*margin
pred_true=sum(testdata.pred_click)
totalcost=pred_true*cost
profit=revenue-totalcost
profit

#define a function to be the scoring (predicted profit on the test dataset)
from sklearn.metrics import make_scorer
def pred_profit(y_true,y_pred):
    d = {'label': y_true, 'model': y_pred[:,1]}
    df = pd.DataFrame(data=d)
    traindata2=df
    traindata2['pred_click']=1
    traindata2.loc[traindata2['model']/2<breakeven_rate,'pred_click']=0
    tp=traindata2.loc[traindata2.pred_click==1].loc[traindata2.label==1]
    pred_true=sum(traindata2.pred_click)
    pred_true_rate=pred_true/len(traindata2)
    send_number=total*pred_true_rate
    adj_response_rate=len(tp)/pred_true/2
    exp_buyers=adj_response_rate*send_number
    totalcost=send_number*cost
    exp_profit=exp_buyers*margin-totalcost
    return exp_profit

score=make_scorer(pred_profit,greater_is_better=True)
# Create the parameter grid: gbm_param_grid
rf_param_grid = {
    'n_estimators': range(100,301,50), ### how to set n_estimators?
    'max_depth': range(2,10)
}

# Instantiate the regressor: gbm
clf = RandomForestClassifier()

# Perform grid search: grid_auc
randomized_clf = RandomizedSearchCV(
    param_distributions=rf_param_grid,estimator=clf,scoring=score,n_iter=50,cv=5,verbose=1
)

# Fit grid_mse to the data
randomized_clf.fit(X_train,y_train)